In [2]:
from agents import Agent, WebSearchTool, trace, gen_trace_id, Runner
from agents.model_settings import ModelSettings
from dataclasses import dataclass, field
from typing import List, Optional
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from IPython.display import display, Markdown

In [3]:
load_dotenv(override=True)

True

In [4]:
instructions="""
    You are an expert in startup competitor research.
    Given a startup idea and target market, find upto or less than 5 competitors.
    Use the WebSearchTool to gather data.
    """

@dataclass
class Competitor:
    name: str = Field(description="The name of the competitor")
    description: str = Field(description="Short 2-3 sentence description of the product")
    website: Optional[str] = Field(website="Website of the product if present")
    strengths: List[str] = Field(description="List of strenghts which standout in the product")
    weaknesses: List[str] = Field(description="List of weaknesses in the product")
    pricing: Optional[str] = Field(description="Pricing of the product if present")
    funding: Optional[str] = Field(description="Funding info of the competitor if present")
    market_position: Optional[str] = Field(description="Market share / position")

@dataclass
class CompetitorAnalysisResult:
    industry: str
    competitors: List[Competitor] = Field(description="List of competitors for the given business idea")
    insights: Optional[str] = None


competitor_agent = Agent(
    name="competitor_analysis_agent",
    tools=[WebSearchTool(search_context_size="low")],
    instructions=instructions,
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
    output_type=CompetitorAnalysisResult
)

In [ ]:
message = "AI Journalling app"

with trace("competition_research"):
    competitor_data = await Runner.run(competitor_agent, message)
    print(competitor_data.final_output)

In [7]:
instructions="""
You are MarketResearchAgent, an AI agent specialized in researching and validating a business idea’s market potential.
Your role is to gather accurate, up-to-date, and credible market data from reputable sources using the WebSearchTool, and compile it into a structured report.
Your findings should help business stakeholders understand industry conditions, opportunities, risks, and target customer segments.
"""

class MarketResearchOutput(BaseModel):
    market_size: str = Field(
        description="The estimated current market size, expressed in monetary terms or user base."
    )
    growth_rate: str = Field(
        description="The expected annual growth rate of the market, ideally with a percentage."
    )
    key_trends: List[str] = Field(
        description="A list of notable trends currently shaping the market."
    )
    major_players: List[str] = Field(
        description="A list of leading companies or entities in this market."
    )
    barriers_to_entry: List[str] = Field(
        description="A list of significant challenges or obstacles new entrants may face."
    )
    opportunities: List[str] = Field(
        description="A list of promising opportunities for growth or differentiation in this market."
    )
    threats: List[str] = Field(
        description="A list of potential risks or external factors that could negatively affect the market."
    )
# Agent 3: Market Research
market_agent = Agent(
    name="market_research_agent",
    tools=[WebSearchTool(search_context_size="low")],
    instructions=instructions,
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
    output_type=MarketResearchOutput
)

In [ ]:
message = "AI Journalling app"

with trace("market_research"):
    market_data = await Runner.run(market_agent, message)
    print(market_data.final_output)

In [22]:
instructions="""
You are the Financial Estimator Agent.
Your job is to take competitor analysis data and market research data,
and estimate the Total Addressable Market (TAM), Serviceable Available Market (SAM),
Serviceable Obtainable Market (SOM), and potential annual revenues.

Steps:
1. Parse and understand the provided competitor and market datasets.
2. Use reasonable market assumptions (growth rates, adoption %, pricing models).
3. Clearly explain the calculations and assumptions.
4. Return a structured output in the expected schema.
"""

class FinancialEstimationResult(BaseModel):
    TAM: float = Field(
        ...,
        description="Total Addressable Market in USD. Represents the total revenue opportunity available if the product captured 100% of the market."
    )
    SAM: float = Field(
        ...,
        description="Serviceable Available Market in USD. Represents the portion of the TAM that can be targeted based on the business's product scope and geographical reach."
    )
    SOM: float = Field(
        ...,
        description="Serviceable Obtainable Market in USD. Represents the realistic market share the business can capture given competition, pricing, and operational constraints."
    )
    estimated_annual_revenue: float = Field(
        ...,
        description="Projected yearly revenue in USD based on current market share assumptions and pricing models."
    )
    assumptions: str = Field(
        ...,
        description="Key assumptions used in the estimation, including growth rates, pricing strategy, adoption rates, and market conditions."
    )

financial_estimator_agent = Agent(
    name="financial_estimator_agent",
    tools=[],  # No external search tools, works with provided data
    instructions=instructions,
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="none"),
    output_type=FinancialEstimationResult
)

In [ ]:
message = "AI Journalling app"

fin_inputs = {
    "competitor_data": competitor_data,
    "market_data": market_data
}

with trace("financial_estimator"):
    financial_data = await Runner.run(
        financial_estimator_agent,
        str(fin_inputs)
        )
    print(financial_data.final_output)

In [26]:
INSTRUCTIONS = (
    "You are a skilled business analyst and professional writer. "
    "Given the outputs from competitor analysis, market analysis, and financial estimation agents, "
    "write a cohesive business report. "
    "The report should have a clear structure, professional tone, and flow logically. "
    "Avoid redundancy. "
    "Include headings, subheadings, and summary insights."
    "Format the output as a markdown report with collapsible <details> sections for competitor, market data and financial data. Convert all tabular or bullet data into proper markdown tables or lists before embedding them."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [ ]:
message = "AI Journalling app"

writer_inputs = {
    "competitor_data": competitor_data,
    "market_data": market_data,
    "financial_estimation_data": financial_data
}

with trace("report_writer"):
    report = await Runner.run(
        writer_agent,
        str(writer_inputs)
        )
    print(report.final_output)

In [ ]:
final: ReportData = report.final_output
display(Markdown(final.markdown_report))